# Thompson Sampling (Guassian reward with 5-arm bandit simulator)

Given the true reward distribution is guassian, we assume the prior distribution is guassian.

$$P(\theta) = \mathcal{N}(\mu_0,\sigma_0^2)$$

Based on the Bayesian rule, posterior is

$$P(\theta|r) = \frac{P(r|\theta)P(\theta)}{P(r)} \propto P(r|\theta)P(\theta)$$

where $r$ is the reward/observation drawn from the bandit simulator, likelihood $P(r|\theta) = \mathcal{N} (r|\hat{\theta}, \sigma^2)$. 

For each iteration, we sample from the posterior $P(\theta|r)$, i.e. $\hat{\theta} \sim \mathcal{N}(\mu_t, \sigma_t)$

To figure out how to update paremeters of sampler distribution, we need to find the expression of posterior distribution. We the approach 2 mentioned in [Mathematics for Machine Learning Cha9 (p278)](https://mml-book.github.io/book/chapter09.pdf) use the log space, 

\begin{align}
\log \mathcal{N}(r|\hat{\theta}, \sigma^2) + \log \mathcal{N}(\hat{\theta}|\mu_t, \sigma_t^2) 
& = -\frac{1}{2} \{\sigma ^ {-2} (r - \hat{\theta}) ^ 2 + \sigma_t^{-2} (\hat{\theta} - \mu_t) ^ 2 \} \\
& = -\frac{1}{2} \{(\sigma_t^{-2} + \sigma ^ {-2}) \hat{\theta}^2 - 2 (\sigma_t^{-2} \mu_t + \sigma^{-2} r) \hat{\theta} \} + const\\
\end{align}

That is, we have:
$$(\mu_{t+1},\sigma_{t+1}) \gets (\frac{\sigma_t^2 r + \sigma^2 \mu_t}{\sigma_t^2 + \sigma^2}, \frac{\sigma_t^2 \sigma^2}{\sigma_t^2 + \sigma^2})$$


In [1]:
# Thompson sampling (5-arm bandit with Guassian rewards)

import numpy as np

def _ts(num_iter, num_arm):

    # initial parameters 

    update_mu = np.zeros((num_arm)) 
    update_sigma = np.ones((num_arm)) * 100
    liki_sigma = np.ones((num_arm)) 

    theta_array = np.zeros((num_arm))

    for t in range(num_iter):

        # sample model
        for i in range(num_arm):
            theta_array[i] = np.random.normal(update_mu[i], update_sigma[i])

        # select and apply action (expection of likilihood is theta)
        max_index = np.argmax(theta_array)
        #print(max_index)
        reward = _simulator(max_index, num_arm)
        #print(reward)

        # update distribution
        tmp_mu = update_mu[max_index]
        tmp_sigma = update_sigma[max_index]

        update_mu[max_index] = (update_mu[max_index] * liki_sigma[max_index] ** 2.0 + update_sigma[max_index] ** 2.0 * reward)/((update_sigma[max_index] ** 2.0) + (liki_sigma[max_index] ** 2.0))
        update_sigma[max_index] = (update_sigma[max_index] ** 2.0) * (liki_sigma[max_index] ** 2.0)/ ((update_sigma[max_index] ** 2.0) + (liki_sigma[max_index] ** 2.0))
        #update_mu[max_index] = update_sigma[max_index] * ( update_mu[max_index]/(tmp_sigma ** 2.0) +  reward/(liki_sigma[max_index] ** 2.0))
        #print(update_mu[max_index])

        '''
        if abs(update_mu[max_index] - tmp_mu) < 10 ** (-11) and abs(update_sigma[max_index] - tmp_sigma)< 10 ** (-11):
            print(t)
            return max_index
        '''
    return max_index

def _simulator(index, num_arm):
    '''
        simulate five-arm bandit.
        input: index (apply x_i arm)
        output: reward 
    '''
    # define true distribution of reward for each arm
    
    #option1
    #true_mu = np.array([1,1.1,1.2,1.3,1.4])
    #true_sigma = np.ones((num_arm)) * 0.01
    
    #option2
    true_mu = np.array([10.0,20.0,30.0,40.0,50.0])
    true_sigma = np.ones((num_arm)) 
    
    reward = np.random.normal(true_mu[index], true_sigma[index])
    return reward   

In [2]:
#for i in range(20):
print((_ts(500,5)))

4


Still, there are some considerations:

* How to decide the initial $\mu_0$ and $\sigma_0$?  
We should pick parameters to span along the $\theta$ as much as possible, which means a large variance should be picked.
* How to choose likelihood $\sigma$? why we don't need to give a prior on it? 
* How to decide the true distribution of the badit simulator?   
The true distribution should ensure the reward have 'difference' among different arms. By difference, I mean if slightly difference in expectation (e.g. [1,1.1,1.2,1.3,1.4]), then we should pick a small variance (e.g. 0.01).